<a href="https://colab.research.google.com/github/musicjae/Dive-into-the-deeplearning/blob/master/linear%20regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l==0.14.1

In [ ]:
%matplotlib inline
from d2l import tensorflow as d2l
import tensorflow as tf
import random

The true parameters generating our dataset will be  𝐰=[2,−3.4]⊤  and  𝑏=4.2 , and our synthetic labels will be assigned according to the following linear model with the noise term  𝜖 :

𝐲=𝐗𝐰+𝑏+𝜖.

𝜖 는 특징, 라벨에 대해 잠재적인 측정 오차를 포착해내는 것으로 간주한다. 우리는 표준 가정이 성립한다고 가정할 것이고 따라서 𝜖 는 0의 평균값을 가진 정규 분포를 따른다. 문제를 쉽게 하기 위해 표준 편차를 0.01 이라고 하자. 

In [ ]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

print('features:', features[0],'\nlabel:', labels[0])


In [ ]:
d2l.set_figsize()
# The semicolon is for displaying the plot only
d2l.plt.scatter(d2l.numpy(features[:, 1]), d2l.numpy(labels), 1);

우리는 위 그림에서 두 그룹 간의 선형적 상관성을 볼 수 있다. ex의 미니배치 고르기를 회상해보자. 이들을 사용하여 모델을 업데이트 해보자. 이 과정은 기계 학습 알고리즘을 학습하기에 근본적인 것이기 때문에 데이터셋을 섞기 위해 효용 함수를 정의하고, 미니배치 내의 그것에 접근하는 것은 가치가 있다.

아래 코드는 이 함수성의 가능한 수행을 보여준다. 이 함수는 배치 사이즈, 특징의 행렬, 라벨의 벡터를 취하고, 미니배치 사이즈를 산출시킨다. 각 미니배치는 특징과 라벨의 튜플로 구성된다.

In [ ]:
def data_iter(batch_size, features, labels):

    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size): # 0~ 사례들 개수로 배치사이즈 보폭으로 iter
        j = tf.constant(indices[i: min(i+batch_size, num_examples)]) # ??
        yield tf.gather(features, j), tf.gather(labels, j)

batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    
    break

반복문이 도는 동안에, 전체 데이터셋이 다 떨어질 때까지 별개의 미니배치를 성공적으로 얻는다. 위에서 수행된 iter이 교육적인 목적에서 좋긴 하지만, 실제에서 이 방식은 비효율적이다. 가령, 우리는 메모리 내의 모든 데이터를 불러오고 많은 랜덤 메모리 접근을 수행하기를 요구받을 수 있다. 

**모델 초기화**  
  
  미니배치 스토캐스틱 경사 하강법으로 모델의 모수를 최적화 하기 이전에, 우리는 먼저 어떤 모수를 가질 필요가 있다. 다음 코드는 표준 분포에서 랜덤 값을 샘플링함으로써 가중치를 초기화하는 것이다.

In [ ]:
w = tf.Variable(tf.random.normal(shape=(2,1), mean=0, stddev=0.01), trainable = True)
b = tf.Variable(tf.zeros(1), trainable = True)


모수를 초기화한 이후에, 다음에 할 일은 그것들이 데이터에 잘 맞을 때까지 업데이트 시키는 것이다. 각각읭 업데이트는 모수에 대한 손실 함수의 그래디언트를 요구한다. 이 그래디언트를 고려할 때, 우리는 손실이 감소하는 방향으로 각 모수를 업데이트 시킬 수 있다.

### 모델 정의하기  
  
  다음으로, 우리는 모델을 그것의 입력값과 모수를 산출값과 관련하여 정의해야 한다. 선형 모델의 산출값을 계산하기 위해, 우리는 단지 입력값 특징 X와 모델 가중치 w의 점곱을 하고 바이어스 b를 각 예에 더하기만 하면 된다는 것을 회상해보자. 아래 Xw는 벡터이고 b는 스칼라이다. 브로드캐스팅 메커니즘은 아래 같이 subsec_bradcasting으로 기술된다. *우리가 벡터와 스칼라를 더할 때, 스칼라는 각 벡터의 component에 더해진다.*

In [ ]:
def linreg(X, w, b):  #@save
    """The linear regression model."""
    return tf.matmul(X, w) + b

### 손실 함수 정의하기  
  
  우리 모델을 업데이트 하는 것은 손실 함수의 그래디언트를 요구하기 때문에, 우리는 손실 함수를 먼저 정의해야 한다. 여기서 우리는 sec_linear_regression으로 기술되는 제곱근 손실 함수를 사용할 것이다. 이 수행에서, 우리는 참값 y를 예측값 y_hat의 shape으로 변환할 필요가 있다. 다음 함수에 의해 반환된 결과는 또한 y_hat shape을 갖는다:

In [ ]:
def squared_loss(y_hat, y):

    return (y_hat - tf.reshape(y, y_hat.shape)) **2 / 2

### 최적화 알고리즘 정의하기  
  
  이 책에서 소개하는 선형 모델이 아닌 다른 어떤 모델도 분석적으로 해결될 수 없다. 따라서 우리는 미니배치 스토캐스틱 경사 하강법을 소개할 기회를 가지려 한다.  
  각 단계에서, 우리의 데이터셋에서 임의적으로 하나의 미니배치를 사용할 때, 우리는 모수에 대한 손실의 그래디언트를 평가할 것이다. 다음으로, 우리는 손실이 감소할 것 같은 방향으로 모수를 업데이트 시킬 것이다. 다음 코드는 모수, 학습률, 배치 사이즈를 고려할 때, 미니배치 스토캐스틱 경사하강법을 적용한다.

In [ ]:
def sgd(params, grads, lr, batch_size):
    for param, grad in zip(params, grads):

        param.assign_sub(lr*grad/batch_size)

### 학습시키기

<h5>학습 코드를 이해하는 것은 중요하다. 왜냐하면 너는 심층학습 커리어를 통틀어 계속 반복해서 거의 비슷한 학습 loop를 볼 것이기 떄문이다.  
각 iter에서, 우리는 트레이닝 ex의 미니배치를 고르고, 이것을 예측 셋에 성립하도록 모델에 통과시킨다. 
 손실을 계산한 뒤, 우리는 각 모수에 대해 그래디언트를 저장하면서 연결망을 통해 역전파를 초기화한다.  
 마지막으로, 우리는 모델 모수를 업데이트 시키기 위해 sgd 최적화 알고리즘을 불러온다.




In [ ]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with tf.GradientTape() as g:
            l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on l with respect to [`w`, `b`]
        dw, db = g.gradient(l, [w, b])
        # Update parameters using their gradient
        sgd([w, b], [dw, db], lr, batch_size)
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_l)):f}')

이 경우에는 우리가 데이터셋을 스스로 종합했기 때문에, 우리는 참인 모수가 무엇인지 정확히 안다. 따라서 우리는 참인 모수와 우리가 학습 루프를 통해 학습한 것을 비교함으로써 학습이 성공적인지 평가할 수 있다. 이들은 거의 비슷하게 나타난다:

In [ ]:
print(f'error in estimating w: {true_w - tf.reshape(w, true_w.shape)}')
print(f'error in estimating b: {true_b - b}')